In [1]:
import torch
import sys

print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"Device: {'CUDA' if torch.cuda.is_available() else 'MPS' if torch.backends.mps.is_available() else 'CPU'}")

Python: 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 11:23:37) [Clang 14.0.6 ]
PyTorch: 2.9.0
Device: MPS


In [2]:
import torch
import os
import sys
import time
import glob
import json
import shutil
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ultralytics import YOLO
from thop import profile
from pathlib import Path

device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f"Using device: {device}")

/Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FlashAttention is not available on this device. Using scaled_dot_product_attention instead.
Using device: mps
Using device: mps


# YOLOv12 Training

Train YOLOv12n model on the dental X-ray dataset.


In [4]:
# Setup YOLOv12 path
project_root = Path('..').resolve() if Path.cwd().name == 'notebooks' else Path('.').resolve()
yolov12_repo_path = project_root / 'yolov12'
sys.path.insert(0, str(yolov12_repo_path))

print(f"✓ YOLOv12 repo: {yolov12_repo_path}")

✓ YOLOv12 repo: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12


In [5]:
# Setup paths
processed = project_root / 'data' / 'processed'
models_dir = project_root / 'models'
models_dir.mkdir(parents=True, exist_ok=True)

# Locate dataset
data_yaml_path = processed / 'dataset.yaml'
if not data_yaml_path.exists():
    raise FileNotFoundError("Dataset not found! Run 01_prepare_dataset.ipynb first.")

data_yaml_path = str(data_yaml_path)
test_images_path = Path(data_yaml_path).parent / "test" / "images"

print(f"✓ Data config: {data_yaml_path}")
print(f"✓ Models dir: {models_dir}")

✓ Data config: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/data/processed/dataset.yaml
✓ Models dir: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/models


## Train Model


In [7]:
model_v12 = YOLO('yolov12n.pt')

# YOLOv12 has compatibility issues with MPS, use CPU instead
device_v12 = 'cpu' if device == 'mps' else device
if device == 'mps':
    print("⚠ YOLOv12 not compatible with MPS, using CPU instead")

print(f"Training YOLOv12n on {device_v12}...")
start_time_v12 = time.time()
results_v12 = model_v12.train(
    data=data_yaml_path,
    epochs=1,  # Change to 50 for production
    imgsz=640,
    batch=8,
    name='yolov12n_dental',
    device=device_v12,
    verbose=True,  # Shows progress bar with epochs
    plots=False,   # Disable plot generation
    save=True
)
train_time_v12 = time.time() - start_time_v12

path_v12_best_weights = results_v12.save_dir / 'weights' / 'best.pt'
print(f"\n✓ Training complete: {train_time_v12:.2f}s")
print(f"✓ Best weights: {path_v12_best_weights}")

⚠ YOLOv12 not compatible with MPS, using CPU instead
Training YOLOv12n on cpu...
New https://pypi.org/project/ultralytics/8.3.225 available 😃 Update with 'pip install -U ultralytics'
New https://pypi.org/project/ultralytics/8.3.225 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.63 🚀 Python-3.13.5 torch-2.9.0 CPU (Apple M1)
Ultralytics 8.3.63 🚀 Python-3.13.5 torch-2.9.0 CPU (Apple M1)
engine/trainer: task=detect, mode=train, model=yolov12n.pt, data=/Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/data/processed/dataset.yaml, epochs=1, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=yolov12n_dental2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ra

train: Scanning /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/data/processed/train/labels.cache... 58 images, 0 backgrounds, 0 corrupt: 100%|██████████| 58/58 [00:00<?, ?it/s]
/Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
train: Scanning /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/data/processed/train/labels.cache... 58 images, 0 backgrounds, 0 corrupt: 100%|██████████| 58/58 [00:00<?, ?it/s]
/Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg

optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 121 weight(decay=0.0), 128 weight(decay=0.0005), 127 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12/runs/detect/yolov12n_dental2
Starting training for 1 epochs...
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 121 weight(decay=0.0), 128 weight(decay=0.0005), 127 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12/runs/detect/yolov12n_dental2
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size

      Epoch    GPU_mem   box_loss   cls_loss   df


        1/1         0G      1.437       4.14      1.255        141        640: 100%|██████████| 8/8 [00:28<00:00,  3.57s/it]
        1/1         0G      1.437       4.14      1.255        141        640: 100%|██████████| 8/8 [00:28<00:00,  3.57s/it]/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:03<00:00,  1.56s/it]

                   all         17        499   0.000354    0.00343   0.000179   5.42e-05



1 epochs completed in 0.009 hours.
Optimizer stripped from /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12/runs/detect/yolov12n_dental2/weights/last.pt, 5.4MB
Optimizer stripped from /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12/runs/detect/yolov12n_dental2/weights/last.pt, 5.4MB
Optimizer stripped from /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12/runs/detect/yolov12n_dental2/weights/best.pt, 5.4MB

Validating /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12/runs/detect/yolov12n_dental2/weights/best.pt...
Ultralytics 8.3.63 🚀 Python-3.13.5 torch-2.9.0 CPU (Apple M1)
Optimizer stripped from /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12/runs/detect/yolov12n_dental2/weights/best.pt, 5.4MB

Validating /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12/runs

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.27s/it]

                   all         17        499   0.000355    0.00343    0.00018   5.43e-05
        cavity_class_0         17        167   0.000714    0.00599   0.000364   0.000182
        cavity_class_1         17        129   0.000705    0.00775   0.000355   3.55e-05
        cavity_class_2         17         65          0          0          0          0
        cavity_class_3         17        138          0          0          0          0
Speed: 0.5ms preprocess, 133.4ms inference, 0.0ms loss, 0.9ms postprocess per image
        cavity_class_0         17        167   0.000714    0.00599   0.000364   0.000182
        cavity_class_1         17        129   0.000705    0.00775   0.000355   3.55e-05
        cavity_class_2         17         65          0          0          0          0
        cavity_class_3         17        138          0          0          0          0
Speed: 0.5ms preprocess, 133.4ms inference, 0.0ms loss, 0.9ms postprocess per image



✓ Training complete: 36.35s
✓ Best weights: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12/runs/detect/yolov12n_dental2/weights/best.pt


## Profile Model


In [8]:
print("Profiling model...")
model_v12_trained = YOLO(path_v12_best_weights)

# Move to CPU for THOP compatibility
model_cpu = model_v12_trained.model.cpu()
dummy_input = torch.randn(1, 3, 640, 640)

flops_v12, params_v12 = profile(model_cpu, inputs=(dummy_input,), verbose=False)
print(f"✓ Parameters: {params_v12/1e6:.2f}M | FLOPs: {flops_v12/1e9:.2f}G")

Profiling model...
✓ Parameters: 2.52M | FLOPs: 2.99G
✓ Parameters: 2.52M | FLOPs: 2.99G


## Measure Inference Time


In [9]:
print("Measuring inference time...")
# Reload model on correct device for inference
model_v12_trained = YOLO(path_v12_best_weights)

test_images = list(test_images_path.glob('*.jpg')) + list(test_images_path.glob('*.png'))

def get_avg_inference_time(model, image_list):
    if len(image_list) == 0:
        return 0
    _ = model(image_list[0], verbose=False)  # Warm-up
    total_time = 0
    for img in image_list:
        start = time.perf_counter()
        _ = model(img, verbose=False)
        total_time += time.perf_counter() - start
    return (total_time / len(image_list)) * 1000

avg_inf_time_v12 = get_avg_inference_time(model_v12_trained, test_images[:10])
print(f"✓ Avg inference: {avg_inf_time_v12:.2f}ms/image ({len(test_images)} test images)")

Measuring inference time...
✓ Avg inference: 130.52ms/image (19 test images)
✓ Avg inference: 130.52ms/image (19 test images)


In [10]:
print("Evaluating on test set...")
metrics_v12 = model_v12_trained.val(split='test', data=data_yaml_path, verbose=False)
map50_v12 = metrics_v12.box.map50
map50_95_v12 = metrics_v12.box.map

print(f"✓ mAP@50: {map50_v12:.4f} | mAP@50-95: {map50_95_v12:.4f}")

Evaluating on test set...
Ultralytics 8.3.63 🚀 Python-3.13.5 torch-2.9.0 CPU (Apple M1)


val: Scanning /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/data/processed/test/labels.cache... 19 images, 0 backgrounds, 0 corrupt: 100%|██████████| 19/19 [00:00<?, ?it/s]
/Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/data/processed/test/labels.cache... 19 images, 0 backgrounds, 0 corrupt: 100%|██████████| 19/19 [00:00<?, ?it/s]
/Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
    

                   all         19        551   0.000388    0.00481   0.000199   3.95e-05
Speed: 1.0ms preprocess, 155.0ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12/runs/detect/val2
✓ mAP@50: 0.0002 | mAP@50-95: 0.0000
Speed: 1.0ms preprocess, 155.0ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/yolov12/runs/detect/val2
✓ mAP@50: 0.0002 | mAP@50-95: 0.0000


## Save Results


In [11]:
print("Saving model and metrics...")
shutil.copy(path_v12_best_weights, models_dir / 'yolov12_best.pt')

metrics_data = {
    'model': 'YOLOv12n',
    'training_time': train_time_v12,
    'params': params_v12 / 1e6,
    'inference_time_ms': avg_inf_time_v12,
    'map50': float(map50_v12),
    'map50_95': float(map50_95_v12)
}

with open(models_dir / 'yolov12_metrics.json', 'w') as f:
    json.dump(metrics_data, f, indent=2)

print(f"\n{'='*60}")
print(f"✓ Model saved: {models_dir}/yolov12_best.pt")
print(f"✓ Metrics saved: {models_dir}/yolov12_metrics.json")
print(f"{'='*60}")

Saving model and metrics...

✓ Model saved: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/models/yolov12_best.pt
✓ Metrics saved: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/models/yolov12_metrics.json


## Visualize Predictions


In [ ]:
print("Visualizing sample predictions...")
sample_images = test_images[:3]
fig, axes = plt.subplots(1, len(sample_images), figsize=(15, 5))
if len(sample_images) == 1:
    axes = [axes]

for idx, img_path in enumerate(sample_images):
    results = model_v12_trained(img_path, verbose=False)
    plotted = results[0].plot()[..., ::-1]  # BGR to RGB
    axes[idx].imshow(plotted)
    axes[idx].axis('off')
    axes[idx].set_title(f"Sample {idx+1}")

plt.tight_layout()
plt.show()
print("\n✅ YOLOv12 training complete!")